In [5]:
import json
from utils.prompter import Prompter
import re
import unicodedata

prompter = Prompter('')
instructions = []

def remove_unicode_characters(text):
    # 将文本中的Unicode字符进行规范化
    normalized_text = unicodedata.normalize("NFKD", text)
    # 删除规范化后的文本中的非ASCII字符
    cleaned_text = normalized_text.encode("ascii", "ignore").decode("ascii")
    return cleaned_text

def remove_links(text):
    # 匹配链接的正则表达式
    pattern = r'http\S+|www\S+'
    # 使用空字符串替换链接
    text_without_links = re.sub(pattern, '', text)
    return text_without_links

output = open('train_data_3_class_clean.jsonl', 'w')
with open('train_data_3_class.jsonl', 'r') as f:
    for line in f:
        data = json.loads(line)
        data_point = {}
        
        
        data['text'] = remove_links(data['text'])
        # print(data['text'])
        data['text'] = remove_unicode_characters(data['text'])
        # print(data['text'])
        # print('======================')
        output.write(json.dumps(data) + '\n')
        data_point["instruction"] = 'What is the sentiment toward Bitcoin in the input sentence? [positive, negative, neutral]'
        data_point["input"] = data['text']
        data_point["output"] = data['label']
        full_prompt = prompter.generate_prompt(
            data_point["instruction"],
            data_point["input"]
        )
        
        instructions.append({'context':full_prompt, 'target':data['label']})
        # break

In [6]:
import pandas as pd

raw_2021 = pd.read_csv('data/tweets_raw_2021.csv',lineterminator='\n')
infer_2021 = pd.read_csv('data/infer_result_2021.csv', lineterminator='\n')

raw_other = pd.read_csv('data/tweets_raw_other.csv',lineterminator='\n')
infer_other = pd.read_csv('data/infer_result_other.csv', lineterminator='\n')


raw_all = pd.concat([raw_2021,raw_other])
infer_all = pd.concat([infer_2021,infer_other])


KeyboardInterrupt: 

In [ ]:
result = pd.concat([raw_all,infer_all[['day','pred','scores','prob']]],axis=1)
# result
# result.to_csv('data/result.csv',index=False)
result.columns

In [ ]:
# 合并推理结果
import pandas as pd
data1 = pd.read_csv('data/result_filter.csv' ,lineterminator='\n')
data2 = pd.read_csv('data/reddit_infer_result_filter.csv',lineterminator='\n')
data1['source'] = 'Twitter'
data2['source'] = 'Reddit'
data2['user_follower_count'] = 99999
print(data1.columns)
print(data2.columns)
# data1 = data1[['id','day','scores','source','label']]
# data2 = data2[['id','day','scores','source','label']]
print(data1.columns)
print(data2.columns)
# data1
res = pd.concat([data1,data2])
print(res.columns)
res['day'] = pd.to_datetime(res['day'])
res = res.sort_values('day', ascending=True)
res.to_csv('data/all_filter.csv',index=False,lineterminator='\n')
# res = res[res['day'].str.find('-')<=0]

# data1 = data1[data1['day'].notnull()]
# data2 = data2[data2['day'].notnull()]
# res = pd.concat([data1,data2])
# print(len(data1)+len(data2))
# res['day'] = pd.to_datetime(res['day'])
# res = res.sort_values('day', ascending=True)
# res.to_csv('infer_result_2020_2023.csv',index=False)












# 处理reddit数据

In [ ]:
import pandas as pd
data = pd.read_csv('data/reddit/bitcoin/submission.csv')
data2 = pd.read_csv('data/socialGrep/bitcoin2023.csv')
data3 = pd.read_csv('data/socialGrep/bitcoin2020-2021.csv')
def process_data(data:pd.DataFrame):
    # print(data.columns)
    if 'submission' in data.columns:
        data['id'] = data['submission']
    data['selftext'] = data['selftext'].fillna('')
    data['selftext'] = data['selftext'].astype(str)
    data['title'] = data['title'].astype(str)
    if 'created' in data.columns:
        data['day'] = pd.to_datetime(data['created'], unit='s').dt.date
    elif 'created_utc' in data.columns:
        data['day'] = pd.to_datetime(data['created_utc'], unit='s').dt.date
    data['selftext'] = data['selftext'].map(lambda text: '' if text in ['[removed]', '[deleted]'] else text)
    data['full_text'] = data['title'] + '\n' + data['selftext']
    data = data.drop_duplicates(['full_text'])
    return data[['id','full_text','day']]


data = process_data(data)
data2 = process_data(data2)
data3 = process_data(data3)
data = pd.concat([data,data2,data3],axis=0)
data = data.sort_values('day', ascending=True)
data = data.drop_duplicates(['full_text'])
data = data.drop_duplicates(['id'])
data.to_csv('data/reddit_raw_2020_2023.csv',index=False)
data

In [ ]:
import pandas as pd
import calendar
test = pd.read_csv('data/tweets_raw_2021.csv')
# # print(test.columns)

test['day'] = pd.to_datetime(test['day'])

# 按照日期进行分组，并统计每个日期的行数
daily_count = test.groupby(test['day'].dt.date).size()

for i in daily_count:
    print(i)
print(daily_count[0:365])

# print(test)


for day in range(calendar.monthrange(2021, 1)[1]):
    print(day)
# res = pd.read_csv('data/reddit_len_2020_2023.csv')
# res
# res = res[(res['full_text'].str.len())<10000]
# res.to_csv('data/reddit_len_2020_2023.csv',index=False)
# test2 = pd.read_csv('data/infer_result_2020_2023.csv', lineterminator='\n')
# print(test2.columns)
# test2


# res = pd.concat([test2[['full_text', 'day','pred','scores']], test[['full_text', 'day','pred','scores']]],axis=0)
# res = res[res['full_text'].notnull()]
# res

In [ ]:
import pandas as pd
data = pd.read_csv('data/socialGrep/bitcoin2023.csv')
data2 = pd.read_csv('data/socialGrep/bitcoin2023.csv')
print(data)




print(data.columns)
data['selftext'] = data['selftext'].astype(str)
data['title'] = data['title'].astype(str)
data['day'] = pd.to_datetime(data['created_utc'], unit='s').dt.date
data['selftext'] = data['selftext'].map(lambda text: '' if text in ['[removed]', '[deleted]'] else text)
data['full_text'] = data['title'] + '\n' + data['selftext']
data = data.drop_duplicates(['full_text'])

data

In [ ]:
import pandas as pd
data = pd.read_csv('data/all_filter.csv',lineterminator='\n')
print(data.columns)
data = data[data['created_at'].notnull()]



Index(['id', 'full_text', 'created_at', 'favorite_count', 'quote_count',
       'reply_count', 'retweet_count', 'user_id', 'user_name',
       'user_screen_name', 'user_description', 'user_friends_count',
       'user_follower_count', 'user_favorite_count', 'user_media_count',
       'cursor', 'period', 'second', 'month', 'day\r', 'day', 'pred', 'scores',
       'prob', 'label', 'source'],
      dtype='object')


In [ ]:
data['second'] = pd.to_datetime(data['created_at'], format='%a %b %d %H:%M:%S +0000 %Y',utc=True)

# print(data['second'])

print(data.at[10000,'created_at'], data.at[10000,'second'])

Fri Jan 10 21:53:43 +0000 2020 2020-01-10 21:53:43+00:00


In [ ]:
import pandas as pd
data = pd.read_csv('data/kaggle/Bitcoin_tweets.csv',lineterminator='\n')
data

C:\Users\67093\AppData\Local\Temp\ipykernel_28760\2793194487.py:2: DtypeWarning: Columns (4,5,6,7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/kaggle/Bitcoin_tweets.csv',lineterminator='\n')


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,DeSota Wilson,"Atlanta, GA","Biz Consultant, real estate, fintech, startups...",2009-04-26 20:05:09,8534.0,7605,4838,False,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,['bitcoin'],Twitter Web App,False
1,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...","['Thursday', 'Btc', 'wallet', 'security']",Twitter for Android,False
2,Tdlmatias,"London, England","IM Academy : The best #forex, #SelfEducation, ...",2014-11-10 10:50:37,128.0,332,924,False,2021-02-10 23:54:48,"Guys evening, I have read this article about B...",NaN,Twitter Web App,False
3,Crypto is the future,NaN,I will post a lot of buying signals for BTC tr...,2019-09-28 16:48:12,625.0,129,14,False,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,"['Bitcoin', 'FX', 'BTC', 'crypto']",dlvr.it,False
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,Europa,Co-founder @RENJERJerky | Forbes 30Under30 | I...,2016-02-03 13:15:55,1249.0,1472,10482,False,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,['BTC'],Twitter Web App,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4689349,TAnotepad,Not financial advice,#Bitcoin | Forever Alone Crypto Trader 😭 | Ana...,2022-11-04 10:18:11,674,1789,27466,False,2023-01-06 17:46:35,"Bitcoin squeeze is SUPER TIGHT, which way will...","['BTC', 'bitcoin', 'Crypto', 'cryptocurrency',...",Twitter for Android,False
4689350,Boba-Feh,South of the North Pole,My backpack has jets. Reformed Necromonger,2008-04-02 21:48:47,79,454,125,False,2023-01-06 17:46:29,Closed #BTC short at 16725. Missed my long pla...,['BTC'],Twitter for Android,False
4689351,Ethereum Yoda,NaN,UP or DOWN...\n.\n.\n.\n.\nPrice matters NOT.,2022-07-24 04:50:18,532,1,0,False,2023-01-06 17:46:22,#Ethereum price update: \n\n#ETH $1263.59 USD\...,"['Ethereum', 'ETH', 'Bitcoin', 'BTC', 'altcoin...",Twitter Web App,False
4689352,Bitcoin Price Ticker,NaN,Tweets the current price of #bitcoin every 5 m...,2022-10-20 07:10:38,83,7,9,False,2023-01-06 17:46:20,1₿ = $16814.7 -0.07%🔻\n\nDetails:\nChange: 🔻-1...,"['bitcoin', 'btc']",BitcoinLivePriceTicker,False


In [ ]:
data['date'] = pd.to_datetime(data['date'], errors='coerce')
data

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,DeSota Wilson,"Atlanta, GA","Biz Consultant, real estate, fintech, startups...",2009-04-26 20:05:09,8534.0,7605,4838,False,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,['bitcoin'],Twitter Web App,False
1,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...","['Thursday', 'Btc', 'wallet', 'security']",Twitter for Android,False
2,Tdlmatias,"London, England","IM Academy : The best #forex, #SelfEducation, ...",2014-11-10 10:50:37,128.0,332,924,False,2021-02-10 23:54:48,"Guys evening, I have read this article about B...",NaN,Twitter Web App,False
3,Crypto is the future,NaN,I will post a lot of buying signals for BTC tr...,2019-09-28 16:48:12,625.0,129,14,False,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,"['Bitcoin', 'FX', 'BTC', 'crypto']",dlvr.it,False
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,Europa,Co-founder @RENJERJerky | Forbes 30Under30 | I...,2016-02-03 13:15:55,1249.0,1472,10482,False,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,['BTC'],Twitter Web App,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4689349,TAnotepad,Not financial advice,#Bitcoin | Forever Alone Crypto Trader 😭 | Ana...,2022-11-04 10:18:11,674,1789,27466,False,2023-01-06 17:46:35,"Bitcoin squeeze is SUPER TIGHT, which way will...","['BTC', 'bitcoin', 'Crypto', 'cryptocurrency',...",Twitter for Android,False
4689350,Boba-Feh,South of the North Pole,My backpack has jets. Reformed Necromonger,2008-04-02 21:48:47,79,454,125,False,2023-01-06 17:46:29,Closed #BTC short at 16725. Missed my long pla...,['BTC'],Twitter for Android,False
4689351,Ethereum Yoda,NaN,UP or DOWN...\n.\n.\n.\n.\nPrice matters NOT.,2022-07-24 04:50:18,532,1,0,False,2023-01-06 17:46:22,#Ethereum price update: \n\n#ETH $1263.59 USD\...,"['Ethereum', 'ETH', 'Bitcoin', 'BTC', 'altcoin...",Twitter Web App,False
4689352,Bitcoin Price Ticker,NaN,Tweets the current price of #bitcoin every 5 m...,2022-10-20 07:10:38,83,7,9,False,2023-01-06 17:46:20,1₿ = $16814.7 -0.07%🔻\n\nDetails:\nChange: 🔻-1...,"['bitcoin', 'btc']",BitcoinLivePriceTicker,False


In [ ]:
data['day'] = data['date'].dt.year.astype(str) + '-' + data['date'].dt.month.astype(str) + '-' + data['date'].dt.day.astype(str)
data['day']

C:\Users\67093\AppData\Local\Temp\ipykernel_1484\1033368574.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['day'] = data['date'].dt.year.astype(str) + '-' + data['date'].dt.month.astype(str) + '-' + data['date'].dt.day.astype(str)


0          2021-2-10
1          2021-2-10
2          2021-2-10
3          2021-2-10
4          2021-2-10
             ...    
4689349     2023-1-6
4689350     2023-1-6
4689351     2023-1-6
4689352     2023-1-6
4689353     2023-1-6
Name: day, Length: 4689288, dtype: object

In [ ]:
data


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,day
0,DeSota Wilson,"Atlanta, GA","Biz Consultant, real estate, fintech, startups...",2009-04-26 20:05:09,8534.0,7605,4838,False,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,['bitcoin'],Twitter Web App,False,2021-2-10
1,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...","['Thursday', 'Btc', 'wallet', 'security']",Twitter for Android,False,2021-2-10
2,Tdlmatias,"London, England","IM Academy : The best #forex, #SelfEducation, ...",2014-11-10 10:50:37,128.0,332,924,False,2021-02-10 23:54:48,"Guys evening, I have read this article about B...",NaN,Twitter Web App,False,2021-2-10
3,Crypto is the future,NaN,I will post a lot of buying signals for BTC tr...,2019-09-28 16:48:12,625.0,129,14,False,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,"['Bitcoin', 'FX', 'BTC', 'crypto']",dlvr.it,False,2021-2-10
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,Europa,Co-founder @RENJERJerky | Forbes 30Under30 | I...,2016-02-03 13:15:55,1249.0,1472,10482,False,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,['BTC'],Twitter Web App,False,2021-2-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4689349,TAnotepad,Not financial advice,#Bitcoin | Forever Alone Crypto Trader 😭 | Ana...,2022-11-04 10:18:11,674,1789,27466,False,2023-01-06 17:46:35,"Bitcoin squeeze is SUPER TIGHT, which way will...","['BTC', 'bitcoin', 'Crypto', 'cryptocurrency',...",Twitter for Android,False,2023-1-6
4689350,Boba-Feh,South of the North Pole,My backpack has jets. Reformed Necromonger,2008-04-02 21:48:47,79,454,125,False,2023-01-06 17:46:29,Closed #BTC short at 16725. Missed my long pla...,['BTC'],Twitter for Android,False,2023-1-6
4689351,Ethereum Yoda,NaN,UP or DOWN...\n.\n.\n.\n.\nPrice matters NOT.,2022-07-24 04:50:18,532,1,0,False,2023-01-06 17:46:22,#Ethereum price update: \n\n#ETH $1263.59 USD\...,"['Ethereum', 'ETH', 'Bitcoin', 'BTC', 'altcoin...",Twitter Web App,False,2023-1-6
4689352,Bitcoin Price Ticker,NaN,Tweets the current price of #bitcoin every 5 m...,2022-10-20 07:10:38,83,7,9,False,2023-01-06 17:46:20,1₿ = $16814.7 -0.07%🔻\n\nDetails:\nChange: 🔻-1...,"['bitcoin', 'btc']",BitcoinLivePriceTicker,False,2023-1-6


In [ ]:
import pandas as pd


data = pd.read_csv('data/tweet_2020_01_full_with_filter.csv',lineterminator='\n')
infer_data = pd.read_csv('data/tweet_2020_01_full_after_filter_infer.csv',lineterminator='\n')
# data = data[data['label'] == 1]
# data.to_csv('data/tweet_2020_01_full_after_filter.csv',index=False)

res = pd.merge(data,infer_data,left_on='id',right_on='id',how='left')



res.to_csv('tweet_2020_01_full_result.csv',index=False)
res.columns

Index(['id', 'full_text', 'created_at', 'favorite_count', 'quote_count',
       'reply_count', 'retweet_count', 'user_id', 'user_name',
       'user_screen_name', 'user_description', 'user_friends_count',
       'user_follower_count', 'user_favorite_count', 'user_media_count',
       'cursor', 'period', 'second', 'month', 'day', 'label', 'pred',
       'scores'],
      dtype='object')

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
    #note: depending on how you installed (e.g., using source code download versus pip install), you may need to import like this:
    #from vaderSentiment import SentimentIntensityAnalyzer

import pandas as pd

data = pd.read_csv('data/all_filter_update_2500.csv',lineterminator='\n')




In [ ]:
analyzer = SentimentIntensityAnalyzer()
for idx,row in data.iterrows():
    vs = analyzer.polarity_scores(row['full_text'])
    vader = f"pos={vs['pos']}\nneg={vs['neg']}\nneu={vs['neu']}\ncompound={vs['compound']}\n"

    data.at[idx,'vader'] = vader
    if idx % 10000 == 0:
        print(idx/len(data))
    

0.0
0.003354177309191754
0.006708354618383508
0.010062531927575263
0.013416709236767016
0.01677088654595877
0.020125063855150525
0.023479241164342277
0.026833418473534033
0.030187595782725785
0.03354177309191754
0.036895950401109295
0.04025012771030105
0.0436043050194928
0.046958482328684555
0.05031265963787631
0.053666836947068065
0.057021014256259814
0.06037519156545157
0.06372936887464332
0.06708354618383508
0.07043772349302684
0.07379190080221859
0.07714607811141035
0.0805002554206021
0.08385443272979384
0.0872086100389856
0.09056278734817735
0.09391696465736911
0.09727114196656086
0.10062531927575262
0.10397949658494438
0.10733367389413613
0.11068785120332789
0.11404202851251963
0.11739620582171138
0.12075038313090314
0.1241045604400949
0.12745873774928665
0.1308129150584784
0.13416709236767016
0.1375212696768619
0.14087544698605367
0.1442296242952454
0.14758380160443718
0.15093797891362892
0.1542921562228207
0.15764633353201243
0.1610005108412042
0.16435468815039594
0.16770886545

In [ ]:
data.to_csv('data/all_filter_update_2500_vader.csv',index=False)

In [ ]:
import pandas as pd
data_path = '../bitcoin_scrapy/update/0711_10000/all_filter_update_0711_mini.csv'
sentiment = pd.read_csv(data_path,dtype={'id':str})
print(sentiment.columns)
print(len(sentiment))

Index(['id', 'full_text', 'pred', 'scores', 'label', 'user_follower_count',
       'favorite_count', 'quote_count', 'reply_count', 'retweet_count', 'day',
       'created_at'],
      dtype='object')
12475979


In [ ]:
sentiment[['id','full_text','pred','scores','label','user_follower_count','favorite_count','quote_count','reply_count','retweet_count','day','created_at']].to_csv('../bitcoin_scrapy/update/0711_10000/all_filter_update_0711_mini.csv',index=False)

In [ ]:
sentiment[['id','pred','scores','label','user_follower_count','favorite_count','quote_count','reply_count','retweet_count','day','created_at']].to_csv('../bitcoin_scrapy/update/0711_10000/all_filter_update_0711_minimun.csv',index=False)